In [1]:
import torch.nn as nn
import torch
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.datasets
import torchvision.transforms as transforms
import torchvision

In [1]:
from DCGAN import *

In [ ]:
dataset=torchvision.datasets.MNIST(root='dataset/',download=True,transform=transforms.ToTensor())
loader=DataLoader(dataset,batch_size=128,shuffle=True)

In [ ]:
criterion=nn.BCEWithLogitsLoss()
epochs=100
z_dim=64
device="cuda"
lr=0.0001
image_dim=28*28*1
batch_size=128



In [ ]:
disc=Discriminator(image_dim).to(device)
gen=Generator(z_dim,image_dim).to(device)
opt_disc=torch.optim.Adam(disc.parameters(),lr=lr)
opt_gen=torch.optim.Adam(gen.parameters(),lr=lr)
writer_fake=SummaryWriter("fake")
writer_real=SummaryWriter("real")


In [ ]:
for epoch in range(epochs):
    for batch_id,(real,_) in enumerate(loader):
        real=real.view(-1,784).to(device)
        noise=torch.randn(batch_size,z_dim).to(device)
        fake=gen(noise)
        disc_real=disc(real).view(-1)
        lossD_real=criterion(disc_real,torch.ones_like(disc_real))
        disc_fake=disc(fake).view(-1)
        lossD_fake=criterion(disc_fake,torch.zeros_like(disc_fake))
        lossD=(lossD_real+lossD_fake)/2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()

        output=disc(fake).view(-1)
        lossG=criterion(output,torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if(batch_id==0):
            with torch.no_grad():
                fake=gen(noise).reshape(-1,1,28,28)
                data=real.reshape(-1,1,28,28)
                img_grid_fake=torchvision.utils.make_grid(fake[:32],normalize=True)
                img_grid_real=torchvision.utils.make_grid(data[:32],normalize=True)
                writer_fake.add_image("MNIST Fake Images",img_grid_fake,global_step=epoch)
                writer_real.add_image("MNIST Real Images",img_grid_real,global_step=epoch)
                print(f"Epoch [{epoch}/{epochs}] Batch {batch_id}/{len(loader)} Loss D: {lossD:.4f}, loss G: {lossG:.4f}")


        